In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import sklearn
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# setting seed
torch.manual_seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)
random.seed(1)

In [12]:
# load data 
train = pd.read_csv("./2021-ai-w10-p1/train.csv")
test = pd.read_csv("./2021-ai-w10-p1/test.csv")
submission = pd.read_csv("./2021-ai-w10-p1/sample_submit.csv")

print(train.info())
print(submission.head())
print(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, Category to 28x28
dtypes: int64(785)
memory usage: 359.3 MB
None
   Id  Category
0   0         0
1   1         0
2   2         0
3   3         0
4   4         0
   Category  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0         5    0    0    0    0    0    0    0    0    0  ...      0      0   
1         0    0    0    0    0    0    0    0    0    0  ...      0      0   
2         3    0    0    0    0    0    0    0    0    0  ...      0      0   
3         3    0    0    0    0    0    0    0    0    0  ...      0      0   
4         6    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3      0      0      0    

In [5]:
# data preprocessing

# data scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

y_train = np.array(train['Category'])
x_train = np.array(train.drop(['Category'],axis=1))
x_test = np.array(test)

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
# set data on Tensor
x_train = torch.Tensor(x_train).to(device)
y_train = torch.LongTensor(y_train).to(device)
x_test = torch.Tensor(x_test).to(device)

print(x_train.shape, x_test.shape)
print(y_train.shape)

torch.Size([60000, 784]) torch.Size([10000, 784])
torch.Size([60000])


In [7]:
# set Tensor on Dataset
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)

In [8]:
# define Model Class
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        
        # setting layer
        self.linear1 = torch.nn.Linear(784,512,bias=True)
        self.linear2 = torch.nn.Linear(512,512,bias=True)
        self.linear3 = torch.nn.Linear(512,256,bias=True)
        self.linear4 = torch.nn.Linear(256,128,bias=True)
        self.linear5 = torch.nn.Linear(128,10,bias=True)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.3)
        
        # setting Model
        self.model = torch.nn.Sequential(self.linear1,self.relu,self.dropout,
                                        self.linear2,self.relu,self.dropout,
                                        self.linear3,self.relu,self.dropout,
                                        self.linear4,self.relu,self.dropout,
                                        self.linear5).to(device)
    def forward(self,x):
        return self.model(x)

model = Model().to(device)

In [9]:
# setting param, optim, cost function,dataloader
from torch.utils.data import DataLoader

lr = 1e-3
batch_size = 500
optim = torch.optim.Adam(model.parameters(), lr=lr)
data_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
epochs = 100
loss = torch.nn.CrossEntropyLoss()

In [10]:
# learning
model.train()
for epoch in range(epochs+1):
    sum_cost = 0
    for x,y in data_loader:
        h = model(x)
        cost = loss(h,y)
        
        optim.zero_grad()
        cost.backward()
        optim.step()
        sum_cost += cost
    if epoch % (epochs/10) == 0 :
        model.eval()
        predict = model(x_train)
        predict = torch.argmax(predict,dim=1)
        acc = predict == y_train
        acc = acc.float().mean().cpu().detach().item()
        print(epoch,sum_cost.item()/len(data_loader),acc)

0 0.6367560704549153 0.9453666806221008
10 0.012918882568677266 0.9964333176612854
20 0.002262988438208898 0.9977499842643738
30 0.005428010722001394 0.9992166757583618
40 0.0042152936259905495 0.9990500211715698
50 0.002635428806145986 0.9998166561126709
60 0.0008558095122377078 0.9998500347137451
70 0.005034990608692169 0.9991000294685364
80 9.719131048768758e-06 1.0
90 1.7754050835113353e-06 1.0
100 6.651992104404296e-07 1.0


In [14]:
with torch.no_grad():
    model.eval()
    predict = model(x_test)
    predict = torch.argmax(predict,dim=1)
    submission['Category'] = predict.cpu().detach()
    print(submission)
    submission.to_csv("submission.csv",index=False)

        Id  Category
0        0         6
1        1         1
2        2         0
3        3         0
4        4         9
...    ...       ...
9995  9995         3
9996  9996         7
9997  9997         2
9998  9998         8
9999  9999         3

[10000 rows x 2 columns]
